In [1]:
from bs4 import BeautifulSoup
import requests
import re
import os
from collections import deque
import json

In [2]:
SAVE_DIR = r"../raw_data"
EXCLUDED_PATH = os.path.join(SAVE_DIR, "excluded_links.json")
DEPTH_LIMIT = 5

In [3]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 11.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

In [4]:
SEED = ["https://vi.wikipedia.org/wiki/L%E1%BB%87_Quy%C3%AAn_(ca_s%C4%A9,_sinh_1981)", 
        "https://vi.wikipedia.org/wiki/Miu_L%C3%AA", 
        "https://vi.wikipedia.org/wiki/H%C3%B2a_Minzy",
        "https://vi.wikipedia.org/wiki/M%E1%BB%B9_Linh",
        "https://vi.wikipedia.org/wiki/Only_C",
        "https://vi.wikipedia.org/wiki/JustaTee",
        "https://vi.wikipedia.org/wiki/Ch%E1%BA%BF_Linh",
        "https://vi.wikipedia.org/wiki/%C4%90%C3%A0m_V%C4%A9nh_H%C6%B0ng",
        "https://vi.wikipedia.org/wiki/Tu%E1%BA%A5n_Ng%E1%BB%8Dc"
        ]

In [5]:
valid_singers = set()

In [6]:
def remove_characters(text):
    if isinstance(text, str):
        # Bỏ cụm [sửa|sửa mã nguồn]
        cleaned = text.replace("[sửa|sửa mã nguồn]", "")
        # Xoá ngoặc và dấu cách/dấu ngoặc kép ở đầu & cuối
        cleaned = re.sub(r'^[\s\(\)\[\]\'"]+|[\s\(\)\[\]\'"]+$', '', cleaned)
        return cleaned.strip()
    return text


In [7]:
def get_years(active_years):
    is_active = False
    if not active_years:
        return None, None

    if isinstance(active_years, str):
        active_years = [active_years]

    start_years = []
    end_years = []

    for period in active_years:
        if not period:
            continue
        p = period.strip()

        # Chuẩn hoá các loại dash thành hyphen thường
        p = re.sub(r'[–—−]', '-', p)

        # 🔹 Lấy tất cả năm và cả từ "nay"
        tokens = re.findall(r'\b(?:19|20)\d{2}\b|\b(?:nay|hiện tại|present|now)\b', p, re.IGNORECASE)

        # Nếu không có token nào, thử kiểm tra dạng đặc biệt "2015-"
        if not tokens:
            if re.search(r'\b(?:19|20)\d{2}\b\s*-\s*$', p):
                start = int(re.search(r'(?:19|20)\d{2}', p).group())
                start_years.append(start)
            continue

        # Xử lý token đầu tiên (năm bắt đầu)
        first = tokens[0]
        if re.match(r'(?:19|20)\d{2}', first):
            start_years.append(int(first))

        # Xử lý token cuối cùng (năm tan rã)
        last = tokens[-1]
        if re.match(r'(?:19|20)\d{2}', last):
            end_years.append(int(last))
        elif re.match(r'(nay|hiện tại|present|now)', last, re.IGNORECASE):
            is_active = True
            # nếu là 'nay' thì không có năm tan rã
            pass
        elif len(tokens) == 1:
            # chỉ có một năm, coi là hoạt động trong năm đó
            end_years.append(int(first))

    if not start_years:
        return None, None

    start = min(start_years)
    end = None if is_active else max(end_years)
    return start, end


In [8]:
def load_excluded_links():
    if os.path.exists(EXCLUDED_PATH):
        with open(EXCLUDED_PATH, "r", encoding="utf-8") as f:
            return set(json.load(f))
    return set()

In [9]:
def save_excluded_links(excluded_links):
    with open(EXCLUDED_PATH, "w", encoding="utf-8") as f:
        json.dump(sorted(list(excluded_links)), f, ensure_ascii=False, indent=2)

In [10]:
def load_crawled_links():
    """Đọc danh sách link đã từng crawl."""
    if os.path.exists("crawled_links.json"):
        with open("crawled_links.json", "r", encoding="utf-8") as f:
            return set(json.load(f))
    return set()

def save_crawled_links(data):
    """Lưu danh sách link đã crawl."""
    with open("crawled_links.json", "w", encoding="utf-8") as f:
        json.dump(list(data), f, ensure_ascii=False, indent=2)


# ====== HÀM CHÍNH ======
def crawl_valid_links(url):
    headers = {'User-Agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')

    stop_headings = ["Chú thích", "Tham khảo", "Liên kết ngoài"]

    content = soup.find('div', id='mw-content-text')
    if not content:
        print("⚠️ Không tìm thấy nội dung chính trong trang!")
        return []

    all_links = []

    for element in content.find_all(['p', 'ul', 'ol', 'div', 'h2', 'h3'], recursive=True):
        if element.name in ['h2', 'h3']:
            heading_text = element.get_text(strip=True)
            if any(stop in heading_text for stop in stop_headings):
                print(f"🛑 Dừng tại mục: {heading_text}")
                break

        for link in element.find_all('a', href=True):
            href = link['href']
            if href.startswith('/wiki/') and not any(x in href for x in [':', '#']):
                full_url = "https://vi.wikipedia.org" + href
                all_links.append(full_url)

    # Loại bỏ trùng lặp trong trang hiện tại
    all_links = list(dict.fromkeys(all_links))
    print(f"🔍 Tìm thấy {len(all_links)} đường dẫn hợp lệ trong trang chính.")

    excluded_links = load_excluded_links()
    crawled_links = load_crawled_links()

    print(f"📂 Bỏ qua {len(excluded_links)} link đã bị loại trước đó...")
    print(f"🧭 Bỏ qua {len(crawled_links)} link đã được crawl trước đó...")

    valid_links = []
    new_excluded = set()
    new_crawled = set()

    keywords = [
        "ca sĩ việt nam", "nam ca sĩ việt nam", "nữ ca sĩ việt nam",
        "ca sĩ gốc việt", "ca sĩ hải ngoại", "nhạc sĩ việt nam",
        "ban nhạc việt nam", "ban nhạc rock việt nam",
        "nhà sản xuất thu âm việt nam", "nhà sản xuất âm nhạc việt nam",
        "nhạc sĩ hòa âm phối khí việt nam", "rapper việt nam"
    ]
    keywords_lower = [k.lower() for k in keywords]

    for link in all_links:
        if link in excluded_links or link in crawled_links:
            print(f"⏩ Bỏ qua (đã loại hoặc đã crawl): {link}")
            continue

        try:
            sub_resp = requests.get(link, headers=headers, timeout=6)
            sub_soup = BeautifulSoup(sub_resp.text, 'html.parser')
            cat_div = sub_soup.find('div', id='mw-normal-catlinks')

            if cat_div:
                cat_text = cat_div.get_text(strip=True).lower()
                if any(k in cat_text for k in keywords_lower):
                    valid_links.append(link)
                    print(f"✅ Giữ lại: {link}")
                else:
                    print(f"❌ Loại bỏ: {link}")
                    new_excluded.add(link)
            else:
                print(f"⚠️ Không tìm thấy danh mục: {link}")
                new_excluded.add(link)

        except Exception as e:
            print(f"⚠️ Lỗi khi truy cập {link}: {e}")
            new_excluded.add(link)

        # Dù giữ hay loại, vẫn đánh dấu là đã crawl
        new_crawled.add(link)

    # Cập nhật dữ liệu
    excluded_links.update(new_excluded)
    crawled_links.update(new_crawled)
    save_excluded_links(excluded_links)
    save_crawled_links(crawled_links)

    print(f"💾 Đã lưu {len(new_crawled)} link mới vào danh sách đã crawl.")
    print(f"✅ Tổng số link hợp lệ mới: {len(valid_links)}")

    return valid_links

In [11]:
def crawl_singer_info(start_urls, depth_limit=DEPTH_LIMIT):
    singers = []
    visited = set()  # tránh trùng lặp
    queue = deque([(url, depth_limit) for url in start_urls])

    while queue:
        url, depth = queue.popleft()  # lấy phần tử đầu (BFS)
        if url in visited or depth <= 0:
            continue
        visited.add(url)

        try:
            print(f"Crawling {url} (depth={depth})...")
            response = requests.get(url, headers=headers)
            response.raise_for_status()
            soup = BeautifulSoup(response.text, 'html.parser')

            # Trỏ vào info box
            info_box = soup.find("table", {"class": "infobox"})
            if not info_box:
                continue

            info_rows = info_box.find_all("tr")
            singer_info = {}
            singer_info['depth'] = depth
            singer_info['name'] = soup.find("h1", {"id": "firstHeading"}).get_text(strip=True)

            for row in info_rows:
                header = row.find("th")
                data = row.find("td")
                if header and data:
                    key = header.get_text(strip=True)

                    # --- TRƯỜNG HỢP 1: Có <div class="hlist"> ---
                    hlist_div = data.find("div", {"class": "hlist"})
                    if hlist_div:
                        items = [li.get_text(strip=True) for li in hlist_div.find_all("li")]
                        singer_info[key] = items
                        continue

                    # --- TRƯỜNG HỢP 2: Có <ul> ---
                    ul_tag = data.find("ul")
                    if ul_tag:
                        items = [li.get_text(strip=True) for li in ul_tag.find_all("li")]
                        singer_info[key] = items
                        continue

                    # --- TRƯỜNG HỢP 3: Có <br> ---
                    if data.find("br"):
                        parts = [text.strip() for text in data.stripped_strings]
                        singer_info[key] = [p for p in parts if p]
                    else:
                        value = data.get_text(separator=' ', strip=True)
                        singer_info[key] = value

            # --- Thêm các trường bổ sung ---
            singer_info['năm thành lập'], singer_info['năm tan rã'] = get_years(singer_info.get('Năm hoạt động'))
            singer_info['link'] = url
            singer_info['relations'] = []

            singers.append(singer_info)

            # --- Thêm các ca sĩ liên quan vào hàng đợi (nếu còn depth) ---
            if depth - 1 > 0:
                colab_links = crawl_valid_links(url)
                singer_info['collaborated_singers'] = colab_links
                for link in colab_links:
                    if link not in visited:
                        queue.append((link, depth - 1))

        except requests.exceptions.RequestException as e:
            print(f"Error fetching {url}: {e}")

    return singers

In [ ]:
singers = crawl_singer_info(SEED)
print(singers)
#save singer to JSON file


Crawling https://vi.wikipedia.org/wiki/L%E1%BB%87_Quy%C3%AAn_(ca_s%C4%A9,_sinh_1981) (depth=5)...
🛑 Dừng tại mục: Chú thích
🔍 Tìm thấy 194 đường dẫn hợp lệ trong trang chính.
📂 Bỏ qua 0 link đã bị loại trước đó...
🧭 Bỏ qua 0 link đã được crawl trước đó...
❌ Loại bỏ: https://vi.wikipedia.org/wiki/L%E1%BB%87_Quy%C3%AAn
❌ Loại bỏ: https://vi.wikipedia.org/wiki/H%C3%A0_N%E1%BB%99i
❌ Loại bỏ: https://vi.wikipedia.org/wiki/Vi%E1%BB%87t_Nam
❌ Loại bỏ: https://vi.wikipedia.org/wiki/Kinh
❌ Loại bỏ: https://vi.wikipedia.org/wiki/Ca_s%C4%A9
❌ Loại bỏ: https://vi.wikipedia.org/wiki/Pop_rock
❌ Loại bỏ: https://vi.wikipedia.org/wiki/Acoustic
❌ Loại bỏ: https://vi.wikipedia.org/wiki/V-pop
❌ Loại bỏ: https://vi.wikipedia.org/wiki/R%26B
❌ Loại bỏ: https://vi.wikipedia.org/wiki/Nh%E1%BA%A1c_v%C3%A0ng
❌ Loại bỏ: https://vi.wikipedia.org/wiki/Nh%E1%BA%A1c_%C4%91%E1%BB%8F
❌ Loại bỏ: https://vi.wikipedia.org/wiki/T%C3%ACnh_kh%C3%BAc_1954-1975
❌ Loại bỏ: https://vi.wikipedia.org/wiki/Nh%E1%BA%A1c_ti%E1%BB%81

In [ ]:
def identify_relationships(singers)
    for singer in singers:
        for link in singer.get('collaborated_singers', []):
            colab_singer = next((s for s in singers if s['link'] == link), None)
            if colab_singer:
                is_same_genre = False  
                for genre in singer.get("Thể loại", []):
                    if "Thể loại" in colab_singer and genre in colab_singer["Thể loại"]:
                        is_same_genre = True
                        break
                if is_same_genre:
                    singer['relations'].append({
                        'singer_link': link,
                        'type': 'same_genre'
                    })
                    colab_singer['relations'].append({
                        'singer_link': singer['link'],
                        'type': 'same_genre'
                    })
                else:
                    singer['relations'].append({
                        'singer_link': link,
                        'type': 'collaborated'
                    })
                    colab_singer['relations'].append({
                        'singer_link': singer['link'],
                        'type': 'collaborated'
                    })

In [ ]:
identify_relationships(singers)

In [ ]:
# Danh sách toàn bộ các tỉnh/thành phố ở Việt Nam trước xác nhập
list_of_provinces = [
    "An Giang", "Bà Rịa - Vũng Tàu", "Bắc Giang", "Bắc Kạn", "Bạc Liêu", "Bắc Ninh",
    "Bến Tre", "Bình Định", "Bình Dương", "Bình Phước", "Bình Thuận", "Cà Mau",
    "Cần Thơ", "Cao Bằng", "Đà Nẵng", "Đắk Lắk", "Đắk Nông", "Điện Biên", "Đồng Nai",
    "Đồng Tháp", "Gia Lai", "Hà Giang", "Hà Nam", "Hà Nội", "Hà Tĩnh", "Hải Dương",
    "Hải Phòng", "Hậu Giang", "Hòa Bình", "Hưng Yên", "Khánh Hòa", "Kiên Giang",
    "Kon Tum", "Lai Châu", "Lâm Đồng", "Lạng Sơn", "Lào Cai", "Long An", "Nam Định",
    "Nghệ An", "Ninh Bình", "Ninh Thuận", "Phú Thọ", "Phú Yên", "Quảng Bình",
    "Quảng Nam", "Quảng Ngãi", "Quảng Ninh", "Quảng Trị", "Sóc Trăng", "Sơn La",
    "Tây Ninh", "Thái Bình", "Thái Nguyên", "Thanh Hóa", "Thừa Thiên Huế", "Tiền Giang",
    "TP. Hồ Chí Minh", "Trà Vinh", "Tuyên Quang", "Vĩnh Long", "Vĩnh Phúc", "Yên Bái", "Đà Lạt", "Sài Gòn"
]


In [ ]:
for singer in singers:
    if "Quê quán" not in singer or not singer["Quê quán"]:
        sinh_info = singer.get("Sinh", [])
        if isinstance(sinh_info, list):
            sinh_text = " ".join(sinh_info)
        else:
            sinh_text = str(sinh_info)

        # Chuyển về chữ thường để so sánh không phân biệt hoa thường
        sinh_lower = sinh_text.lower()

        matched_province = None
        for province in list_of_provinces:
            if province.lower() in sinh_lower:
                matched_province = province
                break

        if matched_province:
            singer["Quê quán"] = matched_province
            print(f"✅ {singer.get('name', 'Không rõ tên')}: tìm thấy quê quán {matched_province}")
        else:
            print(f"⚠️ {singer.get('name', 'Không rõ tên')}: không xác định được quê quán")

✅ Lệ Quyên (ca sĩ, sinh 1981): tìm thấy quê quán Hà Nội
✅ Hòa Minzy: tìm thấy quê quán Bắc Ninh
✅ Only C: tìm thấy quê quán Đà Nẵng
✅ JustaTee: tìm thấy quê quán Hà Nội
⚠️ Chế Linh: không xác định được quê quán
⚠️ Tuấn Ngọc: không xác định được quê quán
✅ Vũ Thành An: tìm thấy quê quán Nam Định
⚠️ Thái Thịnh (nhạc sĩ): không xác định được quê quán
✅ Lam Phương: tìm thấy quê quán Kiên Giang
✅ Thu Phương: tìm thấy quê quán Hải Phòng
✅ Phương Thanh: tìm thấy quê quán Thanh Hóa
⚠️ Minh Tuyết: không xác định được quê quán
✅ Lê Hiếu: tìm thấy quê quán Hà Nội
✅ Ngô Thụy Miên: tìm thấy quê quán Hải Phòng
✅ Đoàn Chuẩn: tìm thấy quê quán Hải Phòng
⚠️ Nguyễn Văn Thương (nhạc sĩ): không xác định được quê quán
⚠️ Phạm Đình Chương: không xác định được quê quán
✅ Tường Văn: tìm thấy quê quán Hà Nội
⚠️ Lam Trường: không xác định được quê quán
✅ Tuấn Hưng: tìm thấy quê quán Hà Nội
⚠️ Jimmii Nguyễn: không xác định được quê quán
✅ Từ Huy: tìm thấy quê quán Quảng Nam
⚠️ Huỳnh Nhật Tân: không xác định được

In [ ]:
import json
with open('../raw_data/singers.json', 'w', encoding='utf-8') as f:
    json.dump(singers, f, ensure_ascii=False, indent=4)